In [1]:
import pandas as pd
import numpy as np

In [2]:
%run limpieza.ipynb
df_eventos_up_to_01062018 = get_clean_df()

In [3]:
df_features = pd.DataFrame(index=df_eventos_up_to_01062018['person'].unique())
df_features.index.name = 'person'

In [4]:
df_features.shape

(38829, 0)

In [5]:
df_features.head()

""
person
4886f805
ad93850f
0297fc1e
2d681dd8
cccea85e


# Features por device

In [6]:
events_per_device_count = df_eventos_up_to_01062018.groupby('person')['device_type'].value_counts()
events_per_device_count.head()

person    device_type
0008ed71  Computer        2
00091926  Computer       34
00091a7a  Smartphone      1
000ba417  Computer        6
000c79fe  Smartphone      1
Name: device_type, dtype: int64

In [7]:
events_per_device_count= events_per_device_count.unstack().fillna(0)
events_per_device_count.head(10)

device_type,Computer,Smartphone,Tablet
person,,,
0008ed71,2.0,0.0,0.0
00091926,34.0,0.0,0.0
00091a7a,0.0,1.0,0.0
000ba417,6.0,0.0,0.0
000c79fe,0.0,1.0,0.0
000e4d9e,13.0,0.0,0.0
000e619d,5.0,0.0,0.0
001001be,0.0,3.0,0.0
0010e89a,1.0,0.0,0.0


In [8]:
df_features = df_features.merge(events_per_device_count, how='inner', on='person')

In [9]:
del df_features['Tablet']
df_features.head()

,Computer,Smartphone
person,,
4886f805,0.0,1.0
ad93850f,0.0,5.0
0297fc1e,0.0,95.0
2d681dd8,2.0,0.0
cccea85e,22.0,0.0


# Feature vive en Brasil

In [10]:
df_eventos_up_to_01062018['vive en Brasil'] = df_eventos_up_to_01062018['country']=='Brazil'
df_vive_Brasil = df_eventos_up_to_01062018.loc[df_eventos_up_to_01062018['event']=='visited site',['person','vive en Brasil']]

In [11]:
df_vive_Brasil = df_vive_Brasil.drop_duplicates()

In [12]:
df_features = df_features.merge(df_vive_Brasil, how='inner', on='person')

In [13]:
df_features.shape

(38788, 4)

In [14]:
df_features.head()

,person,Computer,Smartphone,vive en Brasil
0,4886f805,0.0,1.0,True
1,ad93850f,0.0,5.0,True
2,0297fc1e,0.0,95.0,True
3,2d681dd8,2.0,0.0,True
4,cccea85e,22.0,0.0,True


# Feature compra por mes 

In [15]:
df_cant_conversiones = df_eventos_up_to_01062018.loc[df_eventos_up_to_01062018['event']=='conversion',['person','timestamp']]
df_cant_conversiones['conv']=1
df_cant_conversiones.head()

,person,timestamp,conv
195,49c19e32,2018-05-18 01:12:20,1
234,49c19e32,2018-05-18 01:05:23,1
724,39df97e0,2018-05-29 07:51:14,1
744,380c0e60,2018-05-29 07:13:30,1
1198,35ee0cc8,2018-05-29 19:31:46,1


In [16]:
df_cant_conversiones['timestamp']= pd.to_datetime(df_cant_conversiones['timestamp'])
df_cant_conversiones.head()

,person,timestamp,conv
195,49c19e32,2018-05-18 01:12:20,1
234,49c19e32,2018-05-18 01:05:23,1
724,39df97e0,2018-05-29 07:51:14,1
744,380c0e60,2018-05-29 07:13:30,1
1198,35ee0cc8,2018-05-29 19:31:46,1


## Enero

In [17]:
df_compras_enero = df_cant_conversiones.loc[(df_cant_conversiones['timestamp'].dt.month==1),:]
df_compras_enero = df_compras_enero.groupby('person').agg({'conv':'sum'})
df_compras_enero.head()

,conv
person,
0008ed71,0
00091926,0
00091a7a,0
000ba417,0
000c79fe,0


In [18]:
df_features = df_features.merge(df_compras_enero, how='inner', on='person')
df_features.head()

,person,Computer,Smartphone,vive en Brasil,conv
0,4886f805,0.0,1.0,True,0
1,ad93850f,0.0,5.0,True,0
2,0297fc1e,0.0,95.0,True,0
3,2d681dd8,2.0,0.0,True,0
4,cccea85e,22.0,0.0,True,0


In [19]:
df_features = df_features.rename(columns={'Computer':'Cant visitas con Computadoras','Smartphone':'Cant visitas con smartphone','conv':'compras Enero'})

In [20]:
df_features.head()

,person,Cant visitas con Computadoras,Cant visitas con smartphone,vive en Brasil,compras Enero
0,4886f805,0.0,1.0,True,0
1,ad93850f,0.0,5.0,True,0
2,0297fc1e,0.0,95.0,True,0
3,2d681dd8,2.0,0.0,True,0
4,cccea85e,22.0,0.0,True,0


## Febrero

In [21]:
df_compras_febrero = df_cant_conversiones.loc[(df_cant_conversiones['timestamp'].dt.month==2),:]
df_compras_febrero = df_compras_febrero.groupby('person').agg({'conv':'sum'})
df_compras_febrero.head()

,conv
person,
0008ed71,0
00091926,0
00091a7a,0
000ba417,0
000c79fe,0


In [22]:
df_features = df_features.merge(df_compras_febrero, how='inner', on='person')
df_features.head()

,person,Cant visitas con Computadoras,Cant visitas con smartphone,vive en Brasil,compras Enero,conv
0,4886f805,0.0,1.0,True,0,0
1,ad93850f,0.0,5.0,True,0,0
2,0297fc1e,0.0,95.0,True,0,0
3,2d681dd8,2.0,0.0,True,0,0
4,cccea85e,22.0,0.0,True,0,0


In [23]:
df_features.rename(columns={'conv':'compras Febrero'}, inplace=True)

df_features.head(10)

,person,Cant visitas con Computadoras,Cant visitas con smartphone,vive en Brasil,compras Enero,compras Febrero
0,4886f805,0.0,1.0,True,0,0
1,ad93850f,0.0,5.0,True,0,0
2,0297fc1e,0.0,95.0,True,0,0
3,2d681dd8,2.0,0.0,True,0,0
4,cccea85e,22.0,0.0,True,0,0
5,4c8a8b93,0.0,20.0,True,0,0
6,1b9f7cf6,0.0,3.0,True,0,0
7,29ebb414,0.0,4.0,True,0,0
8,de8fe91b,2.0,0.0,True,0,0
9,45baf068,0.0,1.0,True,0,0


## Marzo

In [24]:
df_compras_marzo = df_cant_conversiones.loc[(df_cant_conversiones['timestamp'].dt.month==3),:]
df_compras_marzo = df_compras_marzo.groupby('person').agg({'conv':'sum'})
df_features = df_features.merge(df_compras_marzo, how='inner', on='person')
df_features.rename(columns={'conv':'compras Marzo'}, inplace=True)

df_features.head(10)

,person,Cant visitas con Computadoras,Cant visitas con smartphone,vive en Brasil,compras Enero,compras Febrero,compras Marzo
0,4886f805,0.0,1.0,True,0,0,0
1,ad93850f,0.0,5.0,True,0,0,0
2,0297fc1e,0.0,95.0,True,0,0,0
3,2d681dd8,2.0,0.0,True,0,0,0
4,cccea85e,22.0,0.0,True,0,0,0
5,4c8a8b93,0.0,20.0,True,0,0,0
6,1b9f7cf6,0.0,3.0,True,0,0,0
7,29ebb414,0.0,4.0,True,0,0,0
8,de8fe91b,2.0,0.0,True,0,0,0
9,45baf068,0.0,1.0,True,0,0,0


## Abril

In [25]:
df_compras_abril = df_cant_conversiones.loc[(df_cant_conversiones['timestamp'].dt.month==4),:]
df_compras_abril = df_compras_abril.groupby('person').agg({'conv':'sum'})
df_features = df_features.merge(df_compras_abril, how='inner', on='person')
df_features.rename(columns={'conv':'compras Abril'}, inplace=True)


## Mayo

In [26]:
df_compras_mayo = df_cant_conversiones.loc[(df_cant_conversiones['timestamp'].dt.month==5),:]
df_compras_mayo = df_compras_mayo.groupby('person').agg({'conv':'sum'})
df_features = df_features.merge(df_compras_mayo, how='inner', on='person')
df_features.rename(columns={'conv':'compras Mayo'}, inplace=True)

In [27]:
df_features.head()

,person,Cant visitas con Computadoras,Cant visitas con smartphone,vive en Brasil,compras Enero,compras Febrero,compras Marzo,compras Abril,compras Mayo
0,4886f805,0.0,1.0,True,0,0,0,0,0
1,ad93850f,0.0,5.0,True,0,0,0,0,0
2,0297fc1e,0.0,95.0,True,0,0,0,0,0
3,2d681dd8,2.0,0.0,True,0,0,0,0,0
4,cccea85e,22.0,0.0,True,0,0,0,0,0
